In [68]:
%load_ext autoreload
%autoreload 2
import numpy as np
import toolbox
print(dir(toolbox.notebook))
from collections import Counter
from utils import get_hero_from_id
from toolbox.notebook import get_json_lines_in_folder, progress_bar

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
['__builtins__', '__cached__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'fs', 'get_files_in_folders', 'get_json_lines_in_folder', 'json', 'np', 'os', 'plt', 'progress_bar', 'ready_image', 'show_image']


In [69]:
import json

matches = []

with open('./scraper/scrape_results/all_matches_15.json', 'rb') as fileref:
    for line in fileref:
        match = json.loads(line)
        if match.get('type', 'match') == 'match':
            matches.append(match)


In [70]:
# Scout game modes
counter = Counter()
for match in progress_bar(matches):
    counter[match.get('game_mode_name', 'Unknown')] += 1
counter

HBox(children=(IntProgress(value=0, max=16107), HTML(value='')))

Counter({'1 vs 1 Solo Mid': 8,
         '?? Event ??': 1504,
         'Ability Draft': 34,
         'All Pick': 32,
         'All Random': 43,
         'All Random Death Match': 12,
         'Captains Draft': 1,
         'Captains Mode': 151,
         'Least Played': 1,
         'Random Draft': 304,
         'Ranked All Pick': 9530,
         'Single Draft': 1280,
         'Unknown': 3207})

In [71]:
def proper_match(match):
    return match.get('game_mode_name', 'Unknown') in ['Ability Draft',
         'All Pick',
         'All Random',
         'All Random Death Match',
         'Captains Mode',
         'Random Draft',
         'Ranked All Pick',
         'Single Draft']

relevant_games = [x for x in matches if proper_match(x)]
print(f"Found {len(relevant_games)} matches")

Found 11386 matches


In [72]:
# Scout hero picks modes
counter = Counter()
for match in progress_bar(relevant_games):
    for player in match['players']:
        counter[get_hero_from_id(player['hero_id'])['name']] += 1
counter

HBox(children=(IntProgress(value=0, max=11386), HTML(value='')))

Counter({'abaddon': 473,
         'abyssal_underlord': 595,
         'alchemist': 564,
         'ancient_apparition': 815,
         'antimage': 1616,
         'arc_warden': 288,
         'axe': 1938,
         'bane': 548,
         'batrider': 224,
         'beastmaster': 382,
         'bloodseeker': 1614,
         'bounty_hunter': 905,
         'brewmaster': 383,
         'bristleback': 1649,
         'broodmother': 450,
         'centaur': 705,
         'chaos_knight': 897,
         'chen': 131,
         'clinkz': 1365,
         'crystal_maiden': 2146,
         'dark_seer': 312,
         'dark_willow': 808,
         'dazzle': 607,
         'death_prophet': 378,
         'disruptor': 862,
         'doom_bringer': 812,
         'dragon_knight': 783,
         'drow_ranger': 1219,
         'earth_spirit': 328,
         'earthshaker': 1331,
         'elder_titan': 301,
         'ember_spirit': 929,
         'enchantress': 489,
         'enigma': 547,
         'faceless_void': 1956,
       

In [73]:
relevant_games[0]['radiant_win']

True

In [74]:
def build_embedding_for_play(player, match_time, radiant_wins): 
    # Gold
    # GPM
    # Gold Spent
    # Assists
    # Denies
    # Deaths
    # Kills
    # Last Hits
    # Level
    # Hero Damage
    # Hero Healing
    # Tower Damage
    # XP Per Minute
    
    player_is_radiant = player['player_slot'] < 127#if its 127 its radient, over its dire
    
    try:
        return {
            'hero_id': player['hero_id'],
            'match_duration': match_time,
            'stats': {
                'win': int(player_is_radiant == radiant_wins),
                'gpm': player['gold_per_min'],
                'assists': player['assists'],
                'denies': player['denies'],
                'deaths': player['deaths'],
                'kills': player['kills'],
                'last_hits': player['last_hits'],
                'level': player['level'],
                'hero_damage': player.get('hero_damage', -1),
                'hero_healing': player.get('hero_healing', -1),
                'tower_damage': player.get('tower_damage', -1),
                'xp_per_min': player['xp_per_min'],
            }
        }
    except KeyError as e:
        print(player)
        raise e
build_embedding_for_play(relevant_games[0]['players'][0], relevant_games[0]['duration'], relevant_games[0]['radiant_win'])

{'hero_id': 7,
 'match_duration': 2032,
 'stats': {'assists': 11,
  'deaths': 9,
  'denies': 0,
  'gpm': 401,
  'hero_damage': 13500,
  'hero_healing': 0,
  'kills': 5,
  'last_hits': 106,
  'level': 19,
  'tower_damage': 1734,
  'win': 1,
  'xp_per_min': 494}}

In [75]:
time_periods = [33 * 60, 40 * 60, 48 * 60]

def period_count_for_duration(duration):
    for i, time_period in enumerate(time_periods):
        if duration < time_period:
            return i
    return len(time_periods)

In [76]:
list(range(0,len(time_periods) + 1))
{x:[] for x in range(0,len(time_periods) + 1)}

{0: [], 1: [], 2: [], 3: []}

In [77]:
hero_embeddings = {}
#with open('./scraper/scrape_results/all_matches_15.json', 'rb') as fileref:
for match in get_json_lines_in_folder('./scraper/scrape_results/'):
    if match and match.get('type', 'match') == 'match' and proper_match(match):
        time_period = period_count_for_duration(match['duration'])
        for player in match['players']:
            embedding = build_embedding_for_play(player, match['duration'], match['radiant_win'])
            previous = hero_embeddings.get(embedding['hero_id'], {x:[] for x in range(0,len(time_periods) + 1)})
            previous[time_period].append(embedding)
            hero_embeddings[embedding['hero_id']] = previous

HBox(children=(IntProgress(value=0, max=21), HTML(value='')))

In [78]:
from collections import Counter
c = Counter()
for match in get_json_lines_in_folder('./scraper/scrape_results/'):
    if match and match.get('type', 'match') == 'match' and proper_match(match):
        if 'duration' not in match:
            print(match)
            raise ValueError
        c[int(match['duration'] / 60)] += 1
from pprint import pprint
for k, v in c.items():
    print(f"{k}, {v}")

HBox(children=(IntProgress(value=0, max=21), HTML(value='')))


46, 13493
43, 13914
55, 4772
39, 14130
41, 18060
60, 2832
63, 1864
84, 98
38, 13496
49, 9582
47, 11109
32, 12891
78, 230
42, 15980
24, 3551
50, 8354
59, 3570
56, 5357
52, 7156
44, 13319
51, 9455
26, 6198
48, 9711
31, 12962
37, 14408
35, 12444
34, 13340
40, 12736
82, 147
27, 6838
71, 760
28, 7467
33, 12580
58, 3897
57, 4363
54, 5560
62, 2127
45, 10608
29, 8151
5, 220
36, 17490
66, 1623
53, 5980
61, 2840
69, 851
25, 4072
64, 1744
73, 529
67, 1237
19, 1168
4, 143
76, 372
3, 128
22, 2331
65, 1477
72, 605
68, 1002
18, 965
75, 388
30, 7713
15, 477
85, 65
17, 840
23, 2759
16, 612
20, 1321
74, 477
86, 61
96, 27
92, 24
101, 6
79, 180
77, 298
21, 1963
94, 20
83, 104
70, 650
6, 318
80, 155
8, 247
11, 312
10, 263
81, 173
7, 289
0, 390
88, 58
100, 10
14, 436
12, 319
2, 80
13, 350
9, 242
168, 1
87, 59
120, 2
98, 16
91, 42
93, 21
95, 25
89, 48
114, 3
110, 2
90, 49
1, 57
99, 12
108, 3
105, 7
97, 13
103, 10
109, 1
128, 1
106, 6
107, 4
104, 3
102, 8
132, 1
115, 2
119, 2
124, 1
116, 1
123, 1
134, 1
113,

In [79]:
print(f"Total: {len(hero_embeddings[7][0])}")
print(f"Total: {len([x for x in hero_embeddings[7][0] if x['stats'].get('hero_damage', -1) is not -1])}")

Total: 10367
Total: 8164


In [80]:
number_of_periods = len(time_periods) + 1

key_order = list(hero_embeddings[7][0][0]['stats'].keys())

key_count = len(key_order)
total_key_count = key_count * number_of_periods
print(total_key_count)
print(total_key_count * number_of_periods)

48
192


In [81]:
def transform_hero_stats(x):
    x['is_str'] = int(x['A'] == 'Strength')
    x['is_agi'] = int(x['A'] == 'Agility')
    x['is_int'] = int(x['A'] == 'Intelligence')
    del x['A']
    if x['is_str'] == 0 and x['is_agi'] == 0 and x['is_int'] == 0:
        print(x)
        raise Exception
    return x
hero_list = {x['id']: transform_hero_stats(x) for x in json.load(open('./stat_augmented_hero_list.json', 'rb'))}

In [82]:
final_embeddings = {}
for hero_id, embeddings in hero_embeddings.items():
    if hero_id == 0:
        continue
    embedding = hero_list[hero_id]
    for period_id, period_results in embeddings.items():
        for key in key_order:
            embedding[f"{period_id}_{key}"] = np.mean([x['stats'][key] for x in period_results if x['stats'][key] != -1])
        final_embeddings[hero_id] = embedding

In [83]:
with open('./hero_embeddings.json', 'wb') as fileref:
    fileref.write(json.dumps(final_embeddings, indent=4).encode('utf-8'))

In [84]:
means = {}
stds = {}
for key in final_embeddings[86].keys():
    if key == 'HERO':
        continue
    values = [float(v[key]) for k, v in final_embeddings.items()]
    means[key] = np.mean(values)
    stds[key] = np.std(values)

def normalize_values(values):
    def normalize_value(k,v):
        if k == 'HERO' or k == 'id':
            return v
        return (float(v) - means[k])/stds[k]
    return {k: normalize_value(k, v) for k, v in values.items()}
    
normalized_embeddings = {k: normalize_values(v) for k, v in final_embeddings.items()}

In [85]:
with open('./normalized_embeddings.json', 'wb') as fileref:
    fileref.write(json.dumps(normalized_embeddings, indent=4).encode('utf-8'))